# Batch Normalization

In [1]:
# !pip install torch torchvision

## 1. Settings
### 1) Import required libraries

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [3]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", 
                         train = True,
                         download = True, 
                         transform = transforms.ToTensor(), 
                         target_transform = None)
mnist_test = dset.MNIST("./", 
                        train = False, 
                        transform = transforms.ToTensor(), 
                        target_transform = None, 
                        download = True)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size = batch_size, 
                                           shuffle = True,
                                           num_workers = 2,
                                           drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size = batch_size, 
                                          shuffle = False,
                                          num_workers = 2,
                                          drop_last = True)

## 3. Model & Optimizer

### 1) CNN Model

In [7]:
# 입력 데이터를 정규화하는것처럼 연산을 통과한 결과값을 정규화할 수 있습니다.
# 그 다양한 방법 중에 대표적인 것이 바로 "Batch Normalization"이고, 이는 컨볼루션 연산처럼 모델에 한 층으로 구현할 수 있습니다.
# https://pytorch.org/docs/stable/nn.html?highlight=batchnorm#torch.nn.BatchNorm2d

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1),    # 28 x 28
            nn.BatchNorm2d(16),                  # x는 입력으로 들어오는 채널의 개수
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding = 1),   # 28 x 28
            nn.BatchNorm2d(32),                  # x는 입력으로 들어오는 채널의 개수
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                  # 14 x 14
            nn.Conv2d(32, 64, 3, padding = 1),   # 14 x 14
            nn.BatchNorm2d(64),                  # x는 입력으로 들어오는 채널의 개수
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                   # 7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 100),
            nn.BatchNorm1d(100),                 # x는 입력으로 들어오는 채널의 개수
            nn.ReLU(),
            nn.Linear(100, 10)
        )       
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

cuda:0


## 4. Train 

In [9]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)                

tensor(1.5841, device='cuda:0', grad_fn=<NllLossBackward>)


In [10]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[ 0.0556,  0.2693, -0.2139],
          [-0.0226,  0.0357, -0.0172],
          [ 0.3066,  0.2955,  0.1873]]],


        [[[-0.0504, -0.1346, -0.2469],
          [ 0.1661,  0.2076, -0.1568],
          [-0.0391,  0.0209,  0.2910]]],


        [[[-0.2989,  0.1412,  0.1379],
          [ 0.2526,  0.2930, -0.2925],
          [-0.1499, -0.0573, -0.2714]]],


        [[[-0.1404, -0.2705,  0.3326],
          [-0.3311,  0.2925,  0.0854],
          [ 0.3299,  0.1355,  0.0320]]],


        [[[-0.0966, -0.0721,  0.2112],
          [ 0.0626, -0.0115,  0.1219],
          [-0.0309,  0.3042,  0.2729]]],


        [[[ 0.1308,  0.0409,  0.1713],
          [ 0.0615,  0.0224, -0.2331],
          [ 0.1094, -0.1626,  0.2828]]],


        [[[-0.2643,  0.2445,  0.1700],
          [ 0.2480,  0.0694,  0.1343],
          [-0.0469, -0.2383,  0.2494]]],


        [[[ 0.1365,  0.2595, -0.0871],
          [ 0.1545, -0.2466,  0.1340],
          [ 0.0407,  0.2652,  0.0602]]],


        [

## 5. Test

In [11]:
correct = 0
total = 0

# 배치 정규화나 드롭아웃은 학습할 때와 테스트 할 때 다르게 동작하기 때문에, 모델을 evaluation 모드로 바꿔서 테스트해야 합니다.
model.eval()

with torch.no_grad():
  for image, label in test_loader:
      x = image.to(device)
      y_ = label.to(device)

      output = model.forward(x)
      _, output_index = torch.max(output, 1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 92.79847717285156
